# Introduction
This project focuses on analyzing Facebook advertising campaign data to explore how different pre-campaign factors influence performance outcomes. Specifically, I examined the impact of user interest, ad budget (Spent), and age group on key marketing metrics such as Click-Through Rate (CTR), Cost Per Click (CPC), and Conversion Rate.

After extracting meaningful insights from the data through grouping and visualization, I developed a classification model to predict whether a campaign is likely to succeed or fail — using only the features available before the campaign is launched.

I initially used a Random Forest Classifier, then experimented with Logistic Regression in an attempt to improve model interpretability and performance.

By: kyrollos Ashraf

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


# ML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LogisticRegression


In [ ]:
#ctr (Click-Through Rate) = clicks / impressions
#CPC (Cost per Click) = spent / clicks
#Conversion_rate	 = conversions / clicks

## Data Wrangle

In [ ]:
def wrangle (path): 
    df = pd.read_csv(path)

    #create CTR - CPC -  conversion_rate
    df["CTR"] = df["Clicks"]/df["Impressions"]
    df["CPC"] = df["Spent"]/df["Clicks"]
    df["Conversion_rate"]=  df["Approved_Conversion"]/df["Clicks"]
    

    df= df.dropna()
    return df

In [ ]:
df= wrangle("/kaggle/input/facebook-adcampaigns/Facebook_Ad_Campaigns.csv")
print(df.shape)
df.iloc[400:410]

In [ ]:
df_cpa = df[df["Approved_Conversion"] > 0].copy()
df_cpa["CPA"] = df_cpa["Spent"] / df_cpa["Approved_Conversion"]
#If the CPA is too high, this interest is being wasted.

In [ ]:
df["gender"].value_counts()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

# Data Cleaning

In [ ]:
df["age"].value_counts().sort_index().plot(kind="pie")

In [ ]:
df["gender"].value_counts().plot(kind="bar")

In [ ]:
df.duplicated().sum()

# EDA – Exploratory Data Analysis

**Understand ad performance by age group and gender, and address questions such as:**                      
1- Which age group has the best click-through rate?            
2- Is there a significant difference between males and females in CPC?              
3- Does CPC change by age or gender?           

In [ ]:
df[["CTR", "CPC", "Conversion_rate"]].describe()

In [ ]:
# Averages for each according to gender scale
gender_metrics = df.groupby("gender")[["CTR", "CPC", "Conversion_rate"]].mean()
gender_metrics

## visual
Average CTR by Age

Average CPC by Gender

Average Conversion Rate by Age and Gender

In [ ]:
gender_metrics.plot(kind="bar")

In [ ]:
# Averages for each according to age scale
age_metrics = df.groupby("age")[["CTR", "CPC", "Conversion_rate"]].mean()
age_metrics

In [ ]:

sns.barplot(x=age_metrics.index, y=age_metrics["CTR"])
plt.title("Average CTR by Age")
plt.ylabel("CTR")
plt.xlabel("Age Group")
plt.xticks(rotation=45)  # لو فيه زحام في التسميات
plt.show()

In [ ]:
#Average CTR by Age
age_metrics.plot(kind="bar")

In [ ]:
# CTR by Age and Gender
sns.barplot(data=df, x="age", y="CTR", hue="gender")
plt.title("CTR by Age and Gender")
plt.ylabel("CTR")
plt.xlabel("Age Group")
plt.legend(title="Gender");

In [ ]:
# Conversion_rate by Age and Gender
sns.barplot(df, x="age", y="Conversion_rate", hue="gender")
plt.title("Conversion_rate by Age and Gender")
plt.ylabel("Conversion_rate")
plt.xlabel("Age Group")
plt.legend(title="Gender");

# Campaign Analysis
Understand which campaigns are performing better and which are less efficient.


In [ ]:
campaign_metrics = df.groupby("fb_campaign_id")[["CTR", "CPC", "Conversion_rate", "Spent"]].mean()
campaign_metrics

In [ ]:
high_CTR = campaign_metrics["CTR"].sort_values(ascending=False).head()

high_Spent=campaign_metrics["Spent"].sort_values(ascending=False).head()

high_Conversion_rate = campaign_metrics["Conversion_rate"].sort_values(ascending=False).head()

In [ ]:
high_CTR.plot(kind="bar", color="orange")
plt.title("Top 5 Campaigns by CTR")
plt.xlabel("Campaign ID")
plt.ylabel("CTR")
plt.xticks(rotation=45);

In [ ]:
df.columns

In [ ]:
df[df["fb_campaign_id"]==109857]

In [ ]:
high_Spent.plot(kind="bar", color="orange")
plt.title("Top 5 Campaigns by CTR")
plt.xlabel("Campaign ID")
plt.ylabel("CTR")
plt.xticks(rotation=45);

In [ ]:
# Sort campaigns by highest Spent first, then by highest Conversion Rate
report = campaign_metrics.sort_values(by=["Spent", "Conversion_rate"], ascending=False)
top_5_campaigns = report.head(5)
top_5_campaigns

In [ ]:
report.to_csv("campaign_performance_report.csv", index=True)

# Interest Analysis
Let's find out which "interest" achieves better performance in campaigns in terms of:

In [ ]:
df["interest"].value_counts().sort_index().plot(kind="pie")

In [ ]:
interest_counts = df["interest"].value_counts()
filtered_interest = interest_counts[interest_counts > 35]
filtered_interest.plot(kind="pie")

In [ ]:
(df["age"][df["interest"]==16]).value_counts().plot(kind="bar", ylabel="counts")

In [ ]:
df["interest"].value_counts()[df["interest"].value_counts() > 35]

In [ ]:
df["interest"].value_counts().describe()

In [ ]:
df["interest"].value_counts().plot.box()
plt.ylabel("interest")

# Interests Analysis

In [ ]:
interest_metrics=df.groupby("interest")[["CTR", "CPC", "Conversion_rate", "Spent"]].mean()
interest_metrics.head()

#### Make sure to apply sorting when creating the metrics after groupby, not just during plotting
**True :**

interest_metrics=df.groupby("interest")[["CTR", "CPC", "Conversion_rate", "Spent"]].mean()

interest_metrics["Conversion_rate"].sort_values(ascending=False).head()


**False  :**

df.groupby("interest")["Conversion_rate"].mean().sort_values(ascending=False).head()



In [ ]:
interest_CTR_metrics = interest_metrics["CTR"].sort_values(ascending=False).head(10)
interest_Conversion_rate_metrics = interest_metrics["Conversion_rate"].sort_values(ascending=False).head(10)
interest_Spent_metrics=interest_metrics["Spent"].sort_values(ascending=True).head(10)
interest_CPC_metrics=interest_metrics["CPC"].sort_values(ascending=True).head(10)

In [ ]:
df_cpa["CPA"].min(), df_cpa["CPA"].max()

In [ ]:
df["interest"].sort_values(ascending=True)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 8))  # 2 صف × 2 عمود

# 1. CTR by Interest
interest_CTR_metrics.plot(kind="bar", color="skyblue", ax=axes[0, 0])
axes[0, 0].set_xlabel("Interest")
axes[0, 0].set_ylabel("CTR")
axes[0, 0].set_title("Top 10 Interests with the highest CTR")

# 2. CPC by Interest
interest_CPC_metrics.plot(kind="bar", color="skyblue", ax=axes[0, 1])
axes[0, 1].set_xlabel("Interest")
axes[0, 1].set_ylabel("CPC")
axes[0, 1].set_title("Top 10 Interests with the lowest CPC")

# 3. Conversion_rate by Interest
interest_Conversion_rate_metrics.plot(kind="bar", color="skyblue", ax=axes[1, 0])
axes[1, 0].set_xlabel("Interest")
axes[1, 0].set_ylabel("Conversion Rate")
axes[1, 0].set_title("Top 10 Interests with the highest Conversion Rate")

# 4. CPA by Interest
# the lowest spent with the highest approved_conversion
interest_CPA_metrics= df_cpa.groupby("interest")["CPA"].mean().sort_values(ascending=True).head(10)  

interest_CPA_metrics.plot(kind="bar", color="skyblue", ax=axes[1, 1])
axes[1, 1].set_xlabel("Interest")
axes[1, 1].set_ylabel("CPA")
axes[1, 1].set_title("Top 10 Interests with the lowest CPA")

# تحسين شكل المخطط
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# إعداد البيانات في قائمة
plot_data = [
    ("Top 10 Interests with the highest CTR", interest_CTR_metrics, "CTR", "max"),
    ("Top 10 Interests with the lowest CPC", interest_CPC_metrics, "CPC", "max"),
    ("Top 10 Interests with the highest Conversion Rate", interest_Conversion_rate_metrics, "Conversion Rate", "max"),
    ("Top 10 Interests with the lowest CPA", interest_CPA_metrics, "CPA", "min"),
]

# إعداد الشكل
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# رسم البلوتات باستخدام loop
for ax, (title, data, ylabel, extreme_type) in zip(axes.flatten(), plot_data):
    colors = ["skyblue"] * len(data)
    if extreme_type == "max":
        colors[data.values.argmax()] = "orange"
    else:  # "min"
        colors[data.values.argmin()] = "green"
    
    data.plot(kind="bar", color=colors, ax=ax)
    ax.set_title(title)
    ax.set_xlabel("Interest")
    ax.set_ylabel(ylabel)

# ترتيب العناصر
plt.tight_layout()
plt.show()


# Age Group Analysis

In [ ]:
age_metrics=df.groupby("age")[["CTR", "CPC", "Conversion_rate", "Spent"]].mean().sort_values(by="Conversion_rate", ascending=False)
age_metrics.head()

In [ ]:
fig, axes=plt.subplots(2,2, figsize=(15,8))

# 1- CTR by age
age_metrics["CTR"].plot(kind="bar" , ax=axes[0,0])
axes[0, 0].set_xlabel("age")
axes[0, 0].set_ylabel("CTR")
axes[0, 0].set_title("age with CTR")

# 2- CPC by age
age_metrics["CPC"].plot(kind="bar" , ax=axes[0,1])
axes[0, 1].set_xlabel("age")
axes[0, 1].set_ylabel("CPC")
axes[0, 1].set_title("age with CPC")

# 3. Conversion_rate by Interest
age_metrics["Conversion_rate"] .plot(kind="bar", ax=axes[1, 0])
axes[1, 0].set_xlabel("age")
axes[1, 0].set_ylabel("Conversion Rate")
axes[1, 0].set_title("age with Conversion Rate")

# 4. CPA by Interest
# low CPA = the lowest spent with the highest approved_conversion
Age_CPA_metrics= df_cpa.groupby("age")["CPA"].mean().sort_values(ascending=True).head(10)  

Age_CPA_metrics.plot(kind="bar", ax=axes[1, 1])
axes[1, 1].set_xlabel("age")
axes[1, 1].set_ylabel("CPA")
axes[1, 1].set_title("age with CPA")

plt.tight_layout()


In [ ]:
#استخراج التوصيات النهائية بناءً على التحليلات



اكتر اهتمام له داتا كتير - اشوف اي اكتر فئة عمريةا ختارته جابت محادثات اعلي


In [ ]:
df["interest"].value_counts().head()
# interest 16 have 116 rows

In [ ]:
df_interest_16 = df[df["interest"]==16].groupby("age")[["CTR","CPC","Conversion_rate"]].mean().sort_values(by="Conversion_rate", ascending=False)
df_interest_16
# the best age groyp with conversion rate in interest 16 is 35-39
# the best age groyp with CPC in interest 16 is 30-34
# the best age groyp with CTR in interest 16 is 45-49

In [ ]:
# نجهز لائحة فاضية لتجميع النتائج
summary = []

# ناخد كل interest موجود في الداتا
for i in df["interest"].unique():
    df_i = df[df["interest"] == i]
    grouped = df_i.groupby("age")[["CTR", "CPC", "Conversion_rate"]].mean()

    best_ctr_age = grouped["CTR"].idxmax()
    best_cpc_age = grouped["CPC"].idxmin()
    best_conv_age = grouped["Conversion_rate"].idxmax()

    summary.append({
        "Interest": i,
        "Best Age (CTR)": best_ctr_age,
        "Best Age (CPC)": best_cpc_age,
        "Best Age (Conv Rate)": best_conv_age
    })

# نحولها لداتا فريم
interest_age_summary = pd.DataFrame(summary)
interest_age_summary.head()


In [ ]:
# نجمّع البيانات: متوسط المقاييس لكل interest + age
grouped = df.groupby(["interest", "age"])[["CTR", "CPC", "Conversion_rate"]].mean()

grouped.head()

In [ ]:
grouped["CTR"].groupby("interest").idxmax().head(2)

In [ ]:
# نعيد تنظيم البيانات بحيث كل interest في سطر
best_age_by_interest = pd.DataFrame({
    "Best Age (CTR)": grouped["CTR"].groupby("interest").idxmax().apply(lambda x: x[1]),
    "Best Age (CPC)": grouped["CPC"].groupby("interest").idxmin().apply(lambda x: x[1]),
    "Best Age (Conv Rate)": grouped["Conversion_rate"].groupby("interest").idxmax().apply(lambda x: x[1])
})

best_age_by_interest.reset_index(inplace=True)
best_age_by_interest.head()


# ML
## feature_selection

In [ ]:
df_model= df[["age", "gender", "interest", "Spent","Conversion_rate"]]
df_model = df_model[df_model["Spent"] < 250]
df_model = df_model[df_model["Conversion_rate"] <2]
df_model["Conversion_rate"]=(df_model["Conversion_rate"]>0)

df_model

In [ ]:
df_model["Conversion_rate"].value_counts()

In [ ]:
df_model["Spent"].describe()

In [ ]:
plt.figure(figsize=(8, 1))
sns.boxplot(data=df_model, x="Spent")
plt.title("Distribution of Spent");

## x_train, y_train split

In [ ]:
target = "Conversion_rate"
X=df_model.drop(columns=target)
y=df_model[target]

In [ ]:
X_train, X_test, y_train, y_test= train_test_split( X,y,  test_size=0.2, random_state=42 )
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

## Create model_1 : RandomForest Classification

In [ ]:

categorical_cols = ["age", "gender", "interest"]
numeric_cols = ["Spent"]

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(drop="first"), categorical_cols),
    ("num", StandardScaler(), numeric_cols)
])

model =   make_pipeline(preprocessor,
                      RandomForestClassifier(random_state=42))

model.fit(X_train, y_train)



from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


## Create model_2 :LogisticRegression

In [ ]:
categorical_cols = ["age", "gender", "interest"]
numeric_cols = ["Spent"]

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(drop="first"), categorical_cols),
    ("num", StandardScaler(), numeric_cols)
])


model = make_pipeline(
    preprocessor,
    LogisticRegression()
)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# إدخال القيم يدويًا
input_data = {
    "age": ["30-34"],          # لازم تكون من القيم الأصلية
    "gender": ["F"],           # "M" أو "F"
    "interest": [25],          # أي رقم من الموجود في بياناتك
    "Spent": [50]            
}


input_df = pd.DataFrame(input_data)


pred = model.predict(input_df)

print(f"📈 Conversion Rate ≈ {pred[0]:.1f}")